<a href="https://colab.research.google.com/github/shizoda/education/blob/main/image/scikit-image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🖼️ デジタル画像の基礎

このノートブックは、デジタル画像の基本的な概念を実践的に学ぶためのものです。コンピュータが画像をどのように「見て」いるのか、つまり「数値の集まり」としてどう扱っているのかを理解することを目的とします。

今回は、一枚の画像の中から特徴的な部分（ROI: Region of Interest）を自動で見つけ出し、その部分を構成するピクセルの値を実際に目で見て確認します。

## 📖 このノートブックで学べること
* Pythonを使った画像の読み込み、リサイズ、色空間の変換
* カラー画像がR (Red), G (Green), B (Blue) の3つのチャンネルから成り立つことの理解
* 「関心領域（ROI）」の概念と、その自動探索アルゴリズムの実装
    * 色の鮮やかさ（色彩情報）とコントラスト（輝度情報）を評価指標とする方法を学びます。
* 画像の特定領域を拡大し、ピクセル単位の数値データを可視化する方法

## 🛠️ ツールの使い方
このノートブックは、インタラクティブなUIを備えています。

1.  **ROI（関心領域）の設定**フォームが表示されます。
2.  **自動探索モード**
    * デフォルト（`manual_x = -1`）のままコードを実行すると、プログラムが画像内で最も特徴的だと判断した領域を自動で探し、その座標（X, Y）を出力します。
3.  **手動設定モード**
    * `manual_x`と`manual_y`に0以上の整数値を入力すると、指定した座標を左上としてROIが設定されます。自動探索で見つかった座標や、自分で試したい座標を入力して結果を比較してみましょう。

それでは、実際にコードを実行して、画像に含まれる値を確認してみましょう。


In [ ]:
# @markdown ### ROI（関心領域）の設定
# @markdown `manual_x` か `manual_y` に-1を設定すると自動探索モードになります。
# @markdown 0以上の値を設定すると、その座標がROIの左上隅として手動で設定されます。
# @markdown ---
manual_x = -1  # @param {type:"integer"}
manual_y = -1  # @param {type:"integer"}
manual_width = 7  # @param {type:"integer"}
manual_height = 10 # @param {type:"integer"}

import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from matplotlib.colors import LinearSegmentedColormap

# 画像のダウンロード
url = "https://raw.githubusercontent.com/shizoda/education/refs/heads/main/sample/irasutoya_drink.png"
image_path = "/tmp/downloaded_image.png"

try:
    with urllib.request.urlopen(url) as response, open(image_path, 'wb') as f:
        f.write(response.read())
except Exception as e:
    print(f"画像のダウンロードに失敗しました。エラー: {e}")
    exit()

# 画像の読み込み (アルファチャンネルを保持)
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

if image is None:
    print("画像の読み込みに失敗しました。")
    exit()

# アルファチャンネルがある場合、透明部分を白に変換
if image.shape[2] == 4:
    alpha_channel = image[:,:,3]
    image[alpha_channel == 0] = [255, 255, 255, 255]

# BGR/BGRAからRGBに変換
if image.shape[2] == 4:
    image_rgb_orig = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
else:
    image_rgb_orig = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 画像を128x128にリサイズ
target_size = (128, 128)
image_rgb = cv2.resize(image_rgb_orig, target_size, interpolation=cv2.INTER_AREA)

# グレースケール変換
gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)

# --- ROIの決定ロジック ---
final_x, final_y, final_w, final_h = 0, 0, manual_width, manual_height

if manual_x < 0 or manual_y < 0:
    print("自動モードでROIを探索します...")
    # ▼▼▼ バランス調整用の重み ▼▼▼
    w_color = 3.0
    w_variance = 10.0

    best_x, best_y = -1, -1
    max_combined_score = -1

    scan_width = image_rgb.shape[1] - final_w
    scan_height = image_rgb.shape[0] - final_h

    if scan_width >= 0 and scan_height >= 0:
        for y in range(scan_height):
            for x in range(scan_width):
                roi_rgb = image_rgb[y:y+final_h, x:x+final_w]

                # 1. カラフルさのスコア
                roi_r = roi_rgb[:, :, 0].astype(np.int16)
                roi_g = roi_rgb[:, :, 1].astype(np.int16)
                roi_b = roi_rgb[:, :, 2].astype(np.int16)
                color_diff_score = np.sum(np.abs(roi_r - roi_g)) + \
                                   np.sum(np.abs(roi_g - roi_b)) + \
                                   np.sum(np.abs(roi_b - roi_r))

                # 2. 明るさの分散スコア
                roi_gray = gray[y:y+final_h, x:x+final_w]
                variance_score = np.var(roi_gray)

                # 3. 最終スコア
                combined_score = (w_color * color_diff_score) + (w_variance * variance_score)

                if combined_score > max_combined_score:
                    max_combined_score = combined_score
                    best_x, best_y = x, y

        final_x, final_y = best_x, best_y
        print(f"探索結果: X={final_x}, Y={final_y} にROIを自動設定しました。")
    else:
        print("画像サイズが探索領域より小さいため、自動探索をスキップしました。")

else:
    print(f"手動モード: X={manual_x}, Y={manual_y} にROIを設定します。")
    final_x, final_y = manual_x, manual_y

# --- ここから後続の処理 ---
# 座標が画像の範囲外にならないように補正
final_x = max(0, min(final_x, image_rgb.shape[1] - final_w))
final_y = max(0, min(final_y, image_rgb.shape[0] - final_h))

# 見つかった領域を矩形で囲む
image_with_rect = image_rgb.copy()
cv2.rectangle(image_with_rect, (final_x, final_y), (final_x + final_w, final_y + final_h), (0, 0, 255), 1)

# 拡大する部分の座標
roi_gray = gray[final_y:final_y+final_h, final_x:final_x+final_w]
roi_r = image_rgb[final_y:final_y+final_h, final_x:final_x+final_w, 0]
roi_g = image_rgb[final_y:final_y+final_h, final_x:final_x+final_w, 1]
roi_b = image_rgb[final_y:final_y+final_h, final_x:final_x+final_w, 2]

# (以下、グラフ描画のコードは変更なし)
# ... (ピクセル値プロット用の関数やカスタムカラーマップ定義) ...

# ピクセル値を文字として表示し、格子を描画する汎用関数
def show_channel_with_grid_and_values(ax, data, title, cmap='gray', vmin=0, vmax=255):
    ax.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(title)
    ax.axis("off")
    # 格子を描画
    for i in range(data.shape[0] + 1):
        ax.axhline(i - 0.5, color='white', lw=1)
    for j in range(data.shape[1] + 1):
        ax.axvline(j - 0.5, color='white', lw=1)
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            # テキストの色を濃度値に応じて変更
            if "Red Channel" in title: text_color = "white"  # if data[i, j] <= 230 else "black"
            elif "Blue Channel" in title: text_color = "white"
            else: text_color = "white" if data[i, j] < 128 else "black"
            ax.text(j, i, f"{data[i, j]:3}", ha="center", va="center", color=text_color, fontsize=10)

# カスタムカラーマップを定義
cdict_red = {'red':   ((0.0, 0.0, 0.0), (1.0, 1.0, 1.0)), 'green': ((0.0, 0.0, 0.0), (1.0, 0.0, 0.0)), 'blue':  ((0.0, 0.0, 0.0), (1.0, 0.0, 0.0))}
Red_cmap = LinearSegmentedColormap('Red_cmap', segmentdata=cdict_red, N=256)
cdict_green = {'red':   ((0.0, 0.0, 0.0), (1.0, 0.0, 0.0)), 'green': ((0.0, 0.0, 0.0), (1.0, 1.0, 1.0)), 'blue':  ((0.0, 0.0, 0.0), (1.0, 0.0, 0.0))}
Green_cmap = LinearSegmentedColormap('Green_cmap', segmentdata=cdict_green, N=256)
cdict_blue = {'red':   ((0.0, 0.0, 0.0), (1.0, 0.0, 0.0)), 'green': ((0.0, 0.0, 0.0), (1.0, 0.0, 0.0)), 'blue':  ((0.0, 0.0, 0.0), (1.0, 1.0, 1.0))}
Blue_cmap = LinearSegmentedColormap('Blue_cmap', segmentdata=cdict_blue, N=256)

# --- グラフ描画 ---
# 1. 全体像のプロット
fig = plt.figure(figsize=(8, 7))

ax1 = fig.add_subplot(3, 4, 1)
ax1.imshow(image_rgb)
ax1.set_title(f"Resized ({target_size[0]}x{target_size[1]})")
ax1.axis("off")

ax4 = fig.add_subplot(3, 4, 2)
ax4.imshow(image_rgb[:,:,0], cmap=Red_cmap)
ax4.set_title("Red Channel")
ax4.axis("off")

ax5 = fig.add_subplot(3, 4, 3)
ax5.imshow(image_rgb[:,:,1], cmap=Green_cmap)
ax5.set_title("Green Channel")
ax5.axis("off")

ax6 = fig.add_subplot(3, 4, 4)
ax6.imshow(image_rgb[:,:,2], cmap=Blue_cmap)
ax6.set_title("Blue Channel")
ax6.axis("off")

ax2 = fig.add_subplot(3, 4, 5)
ax2.imshow(gray, cmap='gray')
ax2.set_title("Grayscale Image")
ax2.axis("off")

ax3 = fig.add_subplot(3, 4, 6)
ax3.imshow(image_with_rect)
ax3.set_title("Region of Interest")
ax3.axis("off")

# 拡大している領域の周囲 32x24 画素を切り出し表示
enlarged_w, enlarged_h = 32, 24
enlarged_x = max(0, final_x - (enlarged_w - final_w) // 2)
enlarged_y = max(0, final_y - (enlarged_h - final_h) // 2)
roi_enlarged = image_with_rect[enlarged_y:enlarged_y+enlarged_h, enlarged_x:enlarged_x+enlarged_w]
ax7 = fig.add_subplot(3, 4, 7)
ax7.imshow(roi_enlarged)
ax7.set_title(f"Enlarged Region")
ax7.axis("off")

plt.tight_layout()
plt.show()

# 2. ROIの詳細プロット
fig, axes = plt.subplots(1, 4, figsize=(9, 4))
show_channel_with_grid_and_values(axes[3], roi_b, "Blue Channel (ROI)", cmap=Blue_cmap)
show_channel_with_grid_and_values(axes[2], roi_g, "Green Channel (ROI)", cmap=Green_cmap)
show_channel_with_grid_and_values(axes[1], roi_r, "Red Channel (ROI)", cmap=Red_cmap)
show_channel_with_grid_and_values(axes[0], roi_gray, "Grayscale (ROI)")
plt.tight_layout()
plt.show()

---
## 🎞️ 演習: scikit-image によるフィルタリング

前のセルでは、画像のうち特定の領域のピクセル値を直接確認しました。

ここからは演習として、`scikit-image` というライブラリを使って、Numpy 配列として読み込んだ画像に**フィルタリング処理**を適用してみましょう。`scikit-image` は、Numpy 配列を直接操作することに特化した科学計算・画像処理ライブラリです。


In [ ]:
# --- セル 1: 🚀 ライブラリのインポートと画像のロード ---
# このセルは、演習に必要なライブラリをインポートし、
# 題材となる画像をロードするためのものです。

from skimage import data, transform, color, filters, io, morphology
from skimage.util import img_as_float, img_as_ubyte
import matplotlib.pyplot as plt
import numpy as np

# 演習用に 'astronaut' (カラー) 画像をロードします
# この画像は (512, 512, 3) の形状を持つ uint8 型の Numpy 配列です
try:
    image_color = data.astronaut()

    print(f"画像 'astronaut' を image_color 変数にロードしました。")
    print(f"変数の型 (type): {type(image_color)}")
    print(f"配列の形状 (shape): {image_color.shape}")
    print(f"データ型 (dtype): {image_color.dtype}")
    print(f"最小値: {image_color.min()}, 最大値: {image_color.max()}")

    # --- 画像の表示 ---
    # キャプションが下に来るように、plt.title() ではなく ax.set_xlabel() を使用します
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(image_color)
    ax.set_title("") # 上のタイトルは空に
    ax.set_xlabel("Original Color (image_color)") # 下にキャプション
    ax.axis('off')
    plt.show()

except Exception as e:
    print(f"画像の読み込み中にエラーが発生しました: {e}")

---
### 🎨 2. グレースケール変換

カラー画像（RGBの3チャンネル）は、`shape` が `(高さ, 幅, 3)` となっています。

これを**グレースケール**（白黒の濃淡）画像に変換しましょう。グレースケール画像は「輝度（明るさ）」のみを表す1チャンネルの画像であり、`shape` は `(高さ, 幅)` となります。

`skimage.color.rgb2gray()` 関数は、人間の視覚特性を考慮した以下の計算式（輝度 $Y$）を用いて、3チャンネルを1チャンネルに合成します。

$$Y = 0.2126 \times R + 0.7152 \times G + 0.0722 \times B$$

**演習:**
`image_color`（0-255 の uint8）を `skimage.util.img_as_float()` を使って `[0.0, 1.0]` の float 型に変換し、それを `skimage.color.rgb2gray()` に渡して `image_gray` に格納してください。

In [ ]:
# --- セル 2: 演習 (グレースケール変換) ---

try:
    # 演習 2-1: 元画像を [0, 1] の float 型に変換
    image_float = img_as_float(image_color)

    # ▼▼▼ 演習 2-2: image_float をグレースケールに変換し、image_gray に格納してください ▼▼▼
    # 前のセルで from skimage import data, transform, color, filters, io, morphology と書かれているので、skimage.color はすでに使えます


    # ▲▲▲ 演習ここまで ▲▲▲

    print(f"元の形状: {image_color.shape} -> グレースケール後の形状: {image_gray.shape}")
    print(f"グレースケール後のデータ型: {image_gray.dtype}") # float64 になります

    # --- 比較表示 ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    ax = axes[0]
    ax.imshow(image_color)
    ax.set_title("")
    ax.set_xlabel("Color Image")
    ax.axis('off')

    ax = axes[1]
    ax.imshow(image_gray, cmap='gray')
    ax.set_title("")
    ax.set_xlabel("Grayscale Image (image_gray)")
    ax.axis('off')

    plt.savefig("grayscale_comparison.png")
    plt.show()

except NameError:
    print("エラー: 'image_color' がありません。セル1を先に実行してください。")
except Exception as e:
    print(f"処理中にエラーが発生しました: {e}")

---
### 🌫️ 3. 平滑化 (ガウシアンフィルタ)

`image_gray` には、画像のザラつき（ノイズ）が含まれています。**平滑化**（ぼかし）は、ノイズを軽減し、画像を滑らかにするための処理です。

`skimage.filters.gaussian()` は、ガウス関数（釣鐘状のカーネル）を使い、中心画素に近いほど重みを大きくして周囲の画素と平均化します。

`sigma` パラメータでぼかしの強さ（標準偏差）を指定します。`sigma` が大きいほど、より広範囲の画素が平均化され、ぼかしが強くなります。

**演習:**
`image_gray`（float型）に `filters.gaussian()` を適用し、`sigma=2.0` を指定して `image_smooth` に格納してください。

In [ ]:
# --- セル 3: 演習 (ガウシアンフィルタ) ---
try:
    SIGMA_VALUE = 2.0 # この値を変えてもOK

    # ▼▼▼ 演習 3-1: image_gray にガウシアンフィルタを適用し、image_smooth に格納 ▼▼▼

    # ▲▲▲ 演習ここまで ▲▲▲

    print(f"ガウシアンフィルタ適用 (sigma={SIGMA_VALUE})")

    # --- 比較表示 ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    ax = axes[0]
    ax.imshow(image_gray, cmap='gray')
    ax.set_title("")
    ax.set_xlabel("Grayscale")
    ax.axis('off')

    ax = axes[1]
    ax.imshow(image_smooth, cmap='gray')
    ax.set_title("")
    ax.set_xlabel(f"Smoothed (sigma={SIGMA_VALUE})")
    ax.axis('off')

    plt.savefig("smoothed_astronaut.png")
    plt.show()

except NameError:
    print("エラー: 'image_gray' がありません。セル2を先に実行してください。")
except Exception as e:
    print(f"処理中にエラーが発生しました: {e}")

---
### 🔪 4. 演習 (コード + 記述): エッジ検出 (Sobelフィルタ)

**エッジ検出**は、輝度（明るさ）が急激に変化する部分、すなわち物体の輪郭（エッジ）を見つける処理です。

`skimage.filters.sobel()` は、画像（Numpy 配列）の各画素において、周囲の画素との輝度の差（勾配）を計算します。差が大きい場所（エッジ）ほど値が 1.0 (白) に近くなります。

**演習 (コード):**
`image_gray`（元のグレースケール画像）に `filters.sobel()` を適用し、`image_edge` という変数に格納してください。

In [ ]:
# --- セル 4: 演習 (エッジ検出) ---

try:
    # ▼▼▼ 演習 4-1: image_gray に Sobel フィルタを適用し、image_edge に格納 ▼▼▼


    # ▲▲▲ 演習ここまで ▲▲▲

    print("Sobel エッジ検出結果 (元画像)")

    # --- 比較表示 ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    ax = axes[0]
    ax.imshow(image_gray, cmap='gray')
    ax.set_title("")
    ax.set_xlabel("Grayscale")
    ax.axis('off')

    ax = axes[1]
    ax.imshow(image_edge, cmap='gray')
    ax.set_title("")
    ax.set_xlabel("Edge Detection (Sobel)")
    ax.axis('off')

    plt.savefig("edge_astronaut.png")
    plt.show()

except NameError:
    print("エラー: 'image_gray' がありません。セル2を先に実行してください。")
except Exception as e:
    print(f"処理中にエラーが発生しました: {e}")

---
### ⚫⚪ 5. 演習: しきい値の自動決定（大津の二値化）

`astronaut` の画像はヒストグラムが複雑で、単純な二値化（白黒化）には向きません。
そこで、`skimage.data.camera()` （カメラマン）という、白黒の濃淡がはっきりした画像に切り替えます。

**大津の二値化** (`filters.threshold_otsu`) は、画像全体の輝度ヒストグラムを分析し、「クラス間分散」が最大になる最適なしきい値を自動で決定します。これにより、背景と前景を最もよく分離できます。

**演習:**
`image_camera` をロードした後、`filters.threshold_otsu()` を使ってしきい値 `thresh` を計算し、`image_camera > thresh` という条件式で二値化画像 `binary_camera` を作成してください。

In [ ]:
# --- セル 5: 演習 (大津の二値化) ---

try:
    # 演習用に 'camera' (グレースケール) 画像をロード
    image_camera = data.camera()
    print(f"画像 'camera' をロードしました。 shape: {image_camera.shape}")

    # ▼▼▼ 演習 5-1: image_camera から大津のしきい値(thresh)を計算 ▼▼▼


    # ▼▼▼ 演習 5-2: しきい値 thresh を使って image_camera を二値化 (binary_camera) ▼▼▼


    # ▲▲▲ 演習ここまで ▲▲▲

    print(f"大津のしきい値: {thresh}")

    # --- 比較表示 ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    ax = axes[0]
    ax.imshow(image_camera, cmap='gray')
    ax.set_title("")
    ax.set_xlabel("Camera (Grayscale)")
    ax.axis('off')

    ax = axes[1]
    ax.imshow(binary_camera, cmap='gray')
    ax.set_title("")
    ax.set_xlabel(f"Otsu's Binarization (Thresh={thresh})")
    ax.axis('off')

    plt.savefig("binary_camera.png")
    plt.show()

except Exception as e:
    print(f"処理中にエラーが発生しました: {e}")